# Loop 4 Strategic Analysis: Understanding the Gap

## Current Status
- Best CV: 70.615745 (strict ensemble)
- Target: 68.891380
- Gap: 1.72 points (2.50%)

## Key Questions
1. What's the theoretical minimum score?
2. Which N values have the most room for improvement?
3. What approaches haven't been tried?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

# Load the ensemble submission
df = pd.read_csv('/home/code/experiments/005_ensemble/submission_ensemble_strict.csv')
df['N'] = df['id'].str.split('_').str[0].astype(int)

# Calculate scores per N
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def strip(v):
    return float(str(v).replace('s', ''))

def get_side(xs, ys, degs):
    V = len(TX)
    mnx, mny = 1e300, 1e300
    mxx, mxy = -1e300, -1e300
    for i in range(len(xs)):
        r = degs[i] * math.pi / 180.0
        c, s = math.cos(r), math.sin(r)
        for j in range(V):
            X = c * TX[j] - s * TY[j] + xs[i]
            Y = s * TX[j] + c * TY[j] + ys[i]
            mnx, mxx = min(mnx, X), max(mxx, X)
            mny, mxy = min(mny, Y), max(mxy, Y)
    return max(mxx - mnx, mxy - mny)

scores = {}
for n, g in df.groupby('N'):
    xs = [strip(v) for v in g['x']]
    ys = [strip(v) for v in g['y']]
    ds = [strip(v) for v in g['deg']]
    side = get_side(xs, ys, ds)
    scores[n] = side**2 / n

print(f'Total score: {sum(scores.values()):.6f}')
print(f'Target: 68.891380')
print(f'Gap: {sum(scores.values()) - 68.891380:.6f}')

In [ ]:
# Theoretical minimum: tree area = 0.1775
# Perfect packing efficiency would be: N * tree_area / side^2 = 1
# So minimum side = sqrt(N * tree_area)
# Minimum score per N = N * tree_area / N = tree_area = 0.1775
# Total minimum = 200 * 0.1775 = 35.5

tree_area = 0.1775
theoretical_min_per_n = tree_area
theoretical_min_total = 200 * tree_area

print(f'Tree area: {tree_area}')
print(f'Theoretical minimum per N: {theoretical_min_per_n}')
print(f'Theoretical minimum total: {theoretical_min_total}')
print(f'Current total: {sum(scores.values()):.6f}')
print(f'Gap from theoretical: {sum(scores.values()) - theoretical_min_total:.6f}')
print(f'Efficiency: {theoretical_min_total / sum(scores.values()) * 100:.2f}%')

In [ ]:
# Calculate efficiency per N
efficiencies = {}
for n, score in scores.items():
    # Efficiency = theoretical_min / actual = tree_area / score
    eff = tree_area / score * 100
    efficiencies[n] = eff

# Find worst efficiency N values
worst = sorted(efficiencies.items(), key=lambda x: x[1])[:20]
print('Worst efficiency N values (most room for improvement):')
for n, eff in worst:
    print(f'  N={n}: {eff:.2f}% efficiency, score={scores[n]:.6f}')

In [ ]:
# Calculate how much improvement is needed per N to reach target
current_total = sum(scores.values())
target = 68.891380
needed_reduction = current_total - target

print(f'Current total: {current_total:.6f}')
print(f'Target: {target}')
print(f'Needed reduction: {needed_reduction:.6f}')
print()

# Score breakdown by N range
print('Score breakdown by N range:')
ranges = [(1, 10), (11, 50), (51, 100), (101, 150), (151, 200)]
for start, end in ranges:
    range_score = sum(scores[n] for n in range(start, end+1))
    range_eff = sum(tree_area for _ in range(start, end+1)) / range_score * 100
    print(f'  N={start}-{end}: score={range_score:.4f}, efficiency={range_eff:.2f}%')

In [ ]:
# Compare with baseline to see where ensemble improved
df_baseline = pd.read_csv('/home/code/experiments/001_baseline/submission_best.csv')
df_baseline['N'] = df_baseline['id'].str.split('_').str[0].astype(int)

baseline_scores = {}
for n, g in df_baseline.groupby('N'):
    xs = [strip(v) for v in g['x']]
    ys = [strip(v) for v in g['y']]
    ds = [strip(v) for v in g['deg']]
    side = get_side(xs, ys, ds)
    baseline_scores[n] = side**2 / n

# Find where ensemble improved
improvements = []
for n in range(1, 201):
    if n in scores and n in baseline_scores:
        diff = baseline_scores[n] - scores[n]
        if diff > 1e-8:
            improvements.append((n, diff, baseline_scores[n], scores[n]))

print(f'Ensemble improved {len(improvements)} N values')
print('\nTop 20 improvements:')
for n, diff, base, ens in sorted(improvements, key=lambda x: -x[1])[:20]:
    print(f'  N={n}: improved by {diff:.6f} ({base:.6f} -> {ens:.6f})')

In [ ]:
# Key insight: The ensemble found improvements by picking best configs from multiple sources
# But we're still 1.72 points from target

# What would it take to reach target?
# Option 1: Improve all N by 2.5% - unlikely with local optimization
# Option 2: Find fundamentally better configurations for some N values

# The target score of 68.891380 IS achievable (it's on the leaderboard)
# This means there exist configurations that are ~1.72 points better

# Where could this improvement come from?
# 1. Better tessellation patterns for large N
# 2. Better asymmetric configurations for medium N
# 3. Novel approaches we haven't tried

print('\nAnalysis of improvement potential:')
print(f'Current: {current_total:.6f}')
print(f'Target: {target}')
print(f'Gap: {needed_reduction:.6f} ({needed_reduction/current_total*100:.2f}%)')
print()
print('If we could improve efficiency by 2.5% across all N, we would reach target.')
print('But local optimization only gives ~0.0004 improvement per hour.')
print('We need fundamentally different configurations, not better optimization of existing ones.')